_is an R notebook_

# Spatial Statistics - Assignment 2 (Analysing Spatial Lattice Data)

**Introduction**

This assignment is an opportunity for you to further explore the `R` programming language by working through lattice data.

The key aspects covered:

> 0) Basic Exploratory Data Analysis (EDA) and Exploratory Spatial Data Analysis (ESDA)  
> 1) Moran's I test for spatial autocorrelation  
> 2) Moran's I significance testing through simulation  
> 3) Moran's scatterplot  
> 4) Correlogram  
> 5) Local Indicators of Spatial Autocorrelation (LISA)  
> 6) Ordinary Least Squares Regression  
> 7) Geographically Weighted Regresssion (GWR)

<div class="alert alert-block alert-danger"><b>REQUIRED: </b></div>

**You are required to insert your outputs and any comment into this document.** The document you submit should therefore contain the existing text in addition to:


> *   Plots and other outputs from executing the code chunks  
> *   Discussion of your plots and other outputs as well as conclusions reached.

This should also include any hypotheses and assumptions made as well as factors that may affect your conclusions. </div>

To help you with interpreting your outputs, please consult the following resources:

|            |            | 
|------------|:-----------|
|Chapter 10 - 12:| **Introduction to geographic information systems by Kang-tsung (2011)**. A copy of this is available on Amathuba inside the Resources folder, Study Material & Books |

**GETTING STARTED**

Retrieve and set your working directory

Use the R code chunk below to:

> a) clear your R environment  
> b) print the current working directory  
> c) set your preferred working directory

In [ ]:
###################################################
### Initial settings:
###################################################

rm(list=ls())
getwd()
#setwd("~/student")

# Load the required packages
# We will use different packages to achieve different goals. The following code chunk assumes that the packages have been installed and loads the packages.

Run install.packages("PackageName"), if the package is not installed on your computer (PackageName is the name of the package you want installed)
# Example of how to install a package:
# install.packages("rgdal")
options(scipen = 999) # turn off scientific notation (so p-values are readable)
options(digits = 4)

library(RColorBrewer)
library(classInt)
library(sp)
library(rgdal)
library(maptools)
library(spdep)
library(spgwr)
library(PerformanceAnalytics)


<div class="alert alert-block alert-success"><b>THE DATA: </b> </div>

This assignment is based on the  [Columbus crime data](https://search.r-project.org/CRAN/refmans/RgoogleMaps/html/columbus.html).  

In this practical you will be working with the Columbus crime data. The data set contains information on crime (combined residential burglaries and residential thefts per thousand households) as well as other variables obtained from [Columbus, OH](https://en.wikipedia.org/wiki/Columbus,_Ohio) in 1980. Unit of analysis: 49 neighbourhoods in Columbus, OH.

The variables contained in the dataset are:  
> •	AREA : computed by ArcView  
> •	PERIMETER : computed by ArcView  
> •	COLUMBUS_ : internal polygon ID (ignore)  
> •	COLUMBUS_I : another internal polygon ID (ignore)  
> •	POLYID : yet another polygon ID  
> •	NEIG : neighborhood id value (1-49); conforms to id value used in Spatial Econometrics book.  
> •	HOVAL : housing value (in 1,000dollars)  
> •	INC : household income (in 1,000dollars)  
> •	CRIME : residential burglaries and vehicle thefts per thousand households in the neighborhood  
> •	OPEN : open space in neighborhood  
> •	PLUMB : percentage housing units without plumbing  
> •	DISCBD : distance to CBD  
> •	X : x coordinate (in arbitrary digitizing units, not polygon coordinates)  
> •	Y : y coordinate (in arbitrary digitizing units, not polygon coordinates)  
> •	NSA : north-south dummy (North=1)  
> •	NSB : north-south dummy (North=1)  
> •	EW : east-west dummy (East=1)  
> •	CP : core-periphery dummy (Core=1)  
> •	THOUS : constant=1,000  
> •	NEIGNO : NEIG+1,000, alternative neighborhood id value  

The data comes in the form of a shapefile  in the `spdep` package.  

Analysis will focus on CRIME, HOVAL, INC, OPEN, PLUM, DISCBD, X and Y.

In [ ]:
##############
### Read shapefile
##############
# Several packages to do this
# rgdal::readOGR
# maptools::readShapePoly

# Read data into R
columbus <- readShapePoly(system.file("etc/shapes/columbus.shp", package="spdep")[1])

class(columbus) # Class of object


In [ ]:
slotNames(columbus) # Check the Components of the SpatialPolygonsDataFrame

In [ ]:
str(columbus)  # Full structure of the object

In [ ]:
# Change map projection if necessary
# readshape <- spTransform(readshape, CRS("+init=epsg:2154"))

# Example of reading shapefile and setting projection if working with your own data
#  projection <- "+proj=longlat +ellps=WGS84 +datum=WGS84"
# readshape <-readShapePoly("datashp.shp", proj4string=CRS(projection))
# The gstat package assumes that data are projected,
# i.e. they should not be provided as lattitude/longitude # and have to tell it which vectors are the corrdinates

# The command below specifies the coordinates and changes the dataframe
# to a "SpatialPointDataFrame", which is necessary for variogram modelling

## 0. Exploratory Data Analysis (EDA) and Exploratory Spatial Data Analysis (ESDA)

It is always advisable to **visualise your data** prior to more formal analysis.
EDA and ESDA will help assess:

> •	the key characteristics of the data including possible data errors  
> •	the existence and location of nonrandom local patterns in the data  
> •	the presence of any outliers

Also useful in suggesting pottential associations between variables.
EDA and ESDA cannot _explain_ the observed patterns.

Formal testing of hypothesis should be carried out by means of multivariate regression modelling.


In [ ]:
pacman::p_unload(graphics)
## The following packages are a base install and will not be unloaded:
## graphics
#Extract the data frame from your shapefile
columbusdata <- columbus@data

class(columbusdata) # Class of object


In [ ]:
names(columbusdata) # Names of variables

In [ ]:
str(columbusdata)   # Structure of the object (gives you details of variable types)

<div class="alert alert-block alert-warning"><b>TASK / QUESTIONS: </b> </div>

*   Create a subset data from the columbusdata (select relevant variables and generate summary statistics etc.). Select the numerical variables in columns 7 to 12 and store the subset data into columbusdata_sub variable. Use the summary function to print the summary statistics.

> [double click in this cell and type your answer]

*   What is the mean, maximum and minimum variable for each data set?

> [double click in this cell and type your answer]


*   Print out the correlation matrix and the correlation plot.




> [double click in this cell and type your answer]

In [ ]:
cor(columbusdata_sub) # Correlation
# Correlation plot using PerformanceAnalytic package
chart.Correlation(columbusdata_sub, histogram=TRUE, pch=19)

In [ ]:
# Plot the zones from the Columbus, Ohio data

plot(columbus,col='wheat') # Create a plot of columbus
# Add labels for each of the zones
text(coordinates(columbus),
     labels=as.character(columbus@data$POLYID),
     cex=0.6,font=2, col="darkred")
box(which='outer',lwd=2)


In [ ]:
# Visualising a variable - Binary classification
plot(columbus, col = ifelse(columbus$CRIME > 40, "lightgrey", "red"))

<div class="alert alert-block alert-warning"><b>QUESTION: </b> </div>


*   What conclusion/s can you draw about the data distribution (provide the plot)?

> [double click in this cell and type your answer]

In [ ]:
# Classifying Data using classIntervals()
# Create class breaks
brks.eq = classIntervals(columbus$CRIME, n = 7, style = "equal")
brks.qt = classIntervals(columbus$CRIME, n = 7, style = "quantile")
brks.jk = classIntervals(columbus$CRIME, n = 7, style = "jenks")
# Other style options "fixed", "sd", "pretty", "kmeans", "hclust", "bclust" and "fisher

In [ ]:
# Link the color pallette to the class breaks (categories) using
# findColours(CATEGORIES,PALETTE)
brks.eqcol = findColours(brks.eq,pal)
brks.qtcol = findColours(brks.qt,pal)
brks.jkcol = findColours(brks.jk,pal)


In [ ]:
# Plot with Equal Breaks
plot(columbus,  col=brks.qtcol, border="black")
legend("topleft",leglabs(round(brks.eq$brks,digits=2)), fill=pal, cex=0.8, bty="n")
title(main="Columbus OH: Crime per 1000 households, 1980  \n (Equal breaks)")


In [ ]:
# Plot with Quantile Breaks
plot(columbus,  col=brks.qtcol, border="black")
legend("topleft",leglabs(round(brks.qt$brks,digits=2)), fill=pal, cex=0.8, bty="n")
title(main="Columbus OH: Crime per 1000 households, 1980  \n (Quantile breaks)")

In [ ]:
# Plot with Jenks Breaks
plot(columbus,  col=brks.jkcol, border="black")
legend("topleft",leglabs(round(brks.jk$brks,digits=2)), fill=pal, cex=0.8, bty="n")
title(main="Columbus OH: Crime per 1000 households, 1980  \n (Jenks breaks)", cex=0.5)

<div class="alert alert-block alert-warning"><b>QUESTION: </b> </div>

*   What conclusions can you draw from the data distribution? And What is the difference between the plots?

> [double click in this cell and type your answer]



**Constructing your spatial weight matrix**


In [ ]:
######
# Constructing weight matrix from shapefile
# To deduce neighbourhood structure from spatial polygons by poly2nb
######

columbus.nb = poly2nb(columbus, queen = T)  # Queen's case is the default. To change to Rook's case, set "queen = F i.e  columbus.nb = poly2nb(columbus, queen = F)

#row standardize the weight matrix
columbus.wts = nb2listw(columbus.nb, style="W")
m = length(columbus$CRIME)
s = Szero(columbus.wts)

# Explore the weight matrix
summary(columbus.wts)


In [ ]:
# coordinates function extracts coordinates of the shapefile
columbuscoords <- coordinates(columbus)
plot(columbus)
plot(columbus.wts, columbuscoords, pch=19, cex=0.6, add=TRUE)

<div class="alert alert-block alert-warning"><b>QUESTION / COMMENTS: </b> </div>



*   What is the average number of links? List the number of least and most connected regions. Provide the weight plot

> [double click in this cell and write your answer here]

## 1. Examine Spatial Autocorrelation


Compute Moran's I for the CRIME variable  

**Null Hypothesis**: There is no spatial autocorrelation for the CRIME variable.

**Alt Hypothesis**: There is spatial autocorrelation for the CRIME variable


In [ ]:
moran(columbus$CRIME, columbus.wts, n = m, S0 = s)

moran.test(columbus$CRIME, columbus.wts)
##
##  Moran I test under randomisation
##
## data:  columbus$CRIME
## weights: columbus.wts
##
## Moran I statistic standard deviate = 5.6, p-value = 0.00000001
## alternative hypothesis: greater
## sample estimates:
## Moran I statistic       Expectation          Variance
##          0.500189         -0.020833          0.008689


In [ ]:
# Moran Scatterplot
columbusmp <- moran.plot(columbus$CRIME, columbus.wts,
                         labels = as.character(columbus.wts$POLYID),
                         xlab = "CRIME", ylab = "Lag of CRIME")


<div class="alert alert-block alert-warning"><b>QUESTION / COMMENTS: </b> </div>



*   Testing Significance of Moran's I Statistic under randomization. What conclusion can you draw from the p-value and from the Moran scatterplot?.

> [double click in this cell and write your answer here]

## 4. Spatial Correlograms

The spatial correlogram is also a useful tool for examining if autocorrelation is a local pheonomena (only present among close neighbors) or if it extends over the entire study area (higher order neighbors).

They show how correlated are pairs of spatial observations when you increase the distance (lag) between them. Significant lags are those that do not cross the horizontal line and are strictly above zero.

In [ ]:
columbus_cor5 <- sp.correlogram(columbus.nb, columbus$CRIME, order = 5, method = "I")
columbus_cor5

In [ ]:
plot(columbus_cor5, main = "Correlelogram of CRIME")

<div class="alert alert-block alert-warning"><b>QUESTION / COMMENTS: </b> </div>

*   What conclusion can you draw from the correlelogram plot, post the summary of the results from Spatial correlogram for `columbus$CRIME`.

> [double click in this cell and write your answer here]

## 5. Local Indicators of Spatial Autocorrelation (LISA)

In [ ]:
### Local Moran's I

columbus_locm <- localmoran(columbus$CRIME, columbus.wts)

CRIME <- scale(columbus$CRIME)
columbus$lag_sCRIME <- lag.listw(columbus.wts, CRIME)

In [ ]:
# Identify the Moran plot quadrant for each observation
columbus$quad_sig <- NA
columbus[(columbus$CRIME >= 0 & columbus$lag_sCRIME >= 0) & (columbus_locm[, 5] <= 0.05),
         "quad_sig"] <- 1
columbus[(columbus$CRIME <= 0 & columbus$lag_sCRIME <= 0) & (columbus_locm[, 5] <= 0.05),
         "quad_sig"] <- 2
columbus[(columbus$CRIME >= 0 & columbus$lag_sCRIME <= 0) & (columbus_locm[, 5] <= 0.05),
         "quad_sig"] <- 3
columbus[(columbus$CRIME <= 0 & columbus$lag_sCRIME >= 0) & (columbus_locm[, 5] <= 0.05),
         "quad_sig"] <- 4
columbus[(columbus_locm[, 5] > 0.05), "quad_sig"] <- 5

In [ ]:
#Plot Results
breaks <- seq(1, 5, 1)
labels <- c("High-High", "Low-Low", "High-Low", "Low-High", "Not Signif.")
np <- findInterval(columbus$quad_sig, breaks)
colors <- c("red", "blue", "lightpink", "skyblue2", "white")
plot(columbus, col = colors[np])
mtext("Local Moran's I\n% Crime in Columbus ", cex = 0.7, side = 3,
      line = 1)
legend("topleft", legend = labels, fill = colors, bty = "n",cex = 0.7)

<div class="alert alert-block alert-warning"><b>QUESTIONS / COMMENTS: </b> </div>



*   Paste the summary of `columbus_locm`, `CRIME`, and `columbus$lag_sCRIME` results on your report. Also, provide the plot of results on your report.



> [double click in this cell and write your answer here]

## 7. Geographically weighted regression

For this section, we use the version of the Columbus data available in the `spgwr` package.

**Aim:** Fit a regression model at each location using observations inversely weighted by their distance to the location. This results in one set of coefficients for each observation.

We fit the same model as before: `crime ~ income + housing`.  

<div class="alert alert-block alert-info"><b>NOTE: </b>  </div>

> The x and y values represent the centroids of the census tracts.
> The first step is to determine the kernel bandwidth. The default kernel is Gaussian. The kernel together with the bandwidth specify the matrix of weights.

In [ ]:
library(spgwr)
data(columbus)
names(columbus)

In [ ]:
# GWR with Gauss
crime.bw <- gwr.sel(crime ~ income + housing,
                  data=columbus,
                  coords=cbind(columbus$x, columbus$y))



In [ ]:
Next fit a geographic regression. This is done with the gwr() function.
# Brackets around the code prints the results
(crime.gauss <- gwr(crime ~ income + housing,
                  data=columbus,
                  coords=cbind(columbus$x, columbus$y),
                  bandwidth=crime.bw))


In [ ]:
# Distribution of betas

d <- cbind(crime.gauss$SDF$income,crime.gauss$SDF$housing)

par(mar=c(3,4,2,2))
boxplot(d,xaxt="n",yaxt="n",pars=list(boxwex=0.3))
axis(1,at=1:2,label=c("Income","Housing"))
axis(2,at=seq(-4,2,.2),las=1)
abline(h=0,lty="4343",col="#7E7E7E")
mtext("Beta i",2,line=3)


<div class="alert alert-block alert-warning"><b>QUESTIONS / COMMENTS: </b> </div>



*   Provide your own code that plots the geographic weighted regression (gwr), paste the plotted model and draw a conclusion



> [double click in this cell and discuss the results here]



*   What conclusion can you draw from the boxplot? (paste the plot on your report)



> [double click in this cell and discuss the results here]